<a href="https://colab.research.google.com/github/Ali-mohammadi-design/Prompt_Engineering_and_Machine_Learning/blob/main/Prompt_engineering_lang_chain_agents_Custom_Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Note: In this project we want to make some customized tools to be used by the agent. In this regard, we would use the concept of decorator in python and making new functions. Then we would add the functions to the list of our functions.

Here's a simpler example of a decorator:

def my_decorator(func):
    def wrapper():


        print("Before calling the function")


        func()


        print("After calling the function")


    return wrapper

@my_decorator


def say_hello():


    print("Hello!")

say_hello()


In this example, my_decorator is a function that takes another function func as its argument. It defines a new function wrapper that prints a message before and after calling func. Then, wrapper is returned.

When say_hello() is called, it's actually say_hello passed through my_decorator, so it's equivalent to:

say_hello = my_decorator(say_hello)

Therefore, the output will be:

Before calling the function


Hello!


After calling the function

This demonstrates how the decorator my_decorator modifies the behavior of the say_hello function without changing its implementation.

In [10]:
! pip install langchain
! pip install openai
! pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=8ec067caf7382c6241d351daa890907949005ea83153ea832b29c745fdf719d9
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [3]:
from langchain.agents import load_tools, initialize_agent,AgentType, tool
from langchain.chat_models import ChatOpenAI

Note: Tool function here is a decorator.

Note: We want to ask the LLM who is the coolest guy in the universe. Obviously it answers I do not know. Thus, we want to make a new function and add it to the agent's tools so that when this question is asked it returns my name.

Note: inside the function should be descriptive because the agent would decide to choose the function according to the descrption.

In [5]:
@tool
def coolest_guy(text:str) -> str:
  ''' Returns the name of the coolest person in the universe.
  Expects the input of the empty '' string and return the name of the coolest person in the universe.'''
  return 'Ali'

In [7]:
import os
openAI_API_Key=os.environ['openAI_API_Key']

In [8]:
llm=ChatOpenAI(openai_api_key=openAI_API_Key, temperature=0)

Let's first do it without the coolest_guy() function

In [11]:
tools=load_tools(['wikipedia','llm-math'], llm=llm)

In [13]:
agent=initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [14]:
agent.run('who is the coolest person in the universe?')



> Entering new AgentExecutor chain...
This is a subjective question and cannot be answered definitively.
Action: N/A
Thought: I cannot provide a definitive answer to this question.
Final Answer: N/A

> Finished chain.


'N/A'

Note: Tools is a list. we can add a list to a list with +

In [16]:
tools2=tools+[coolest_guy]

In [17]:
agent2=initialize_agent(tools2, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [18]:
agent2.run('who is the coolest person in the universe?')



> Entering new AgentExecutor chain...
I should use the coolest_guy tool to find out who the coolest person is.
Action: coolest_guy
Action Input: ''
Observation: Ali
Thought:I now know the final answer
Final Answer: Ali

> Finished chain.


'Ali'

Let's provide another example.

In [19]:
agent2.run('what time is it right now?')



> Entering new AgentExecutor chain...
I should use the calculator to find the current time.
Action: Calculator
Action Input: current time calculation
Observation: Answer: 30
Thought:The calculator cannot provide the current time. I should try a different approach. 
Action: N/A
Thought: I should use a different tool to find the current time. 
Action: N/A
Thought: I should check the time on a device or website that displays the current time. 
Action: N/A
Final Answer: I cannot determine the current time with the tools available.

> Finished chain.


'I cannot determine the current time with the tools available.'

In [20]:
from datetime import datetime

In [23]:
str(datetime.now())

'2024-02-21 15:31:24.963713'

In [26]:
@tool
def get_the_current_time(text: str)->str:
  ''' Returns the current time. Use this for any questiosn regarding the current time.
  Input is an empty string '' and the current time is returned in a string format.
  Only use this function for the current time. Ohter time related questions should use another tools'''
  return str(datetime.now())

In [27]:
tools3=tools2+[get_the_current_time]

In [28]:
agent3=initialize_agent(tools3, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [29]:
agent3.run('what time is it right now?')



> Entering new AgentExecutor chain...
I should use the get_the_current_time function for this question.
Action: get_the_current_time
Action Input: ''
Observation: 2024-02-21 15:36:08.813036
Thought:I have the current time now.

Final Answer: The current time is 2024-02-21 15:36:08.813036.

> Finished chain.


'The current time is 2024-02-21 15:36:08.813036.'